# Prediccion de precios de las casas

## Importamos librerias

In [1]:
#Libreria para trabajar con dataframes
import pandas as pd
# Libreria para vectorizar 
import numpy as np
# Libreria para trabajar con paths relativos
import os
# Libreria para usar modelos de machine learning
from sklearn.linear_model import LinearRegression
# Libreria para estandarizar valores numericos, libreria para hacer one hot encoding
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Libreria para imputar valores
from sklearn.impute import SimpleImputer
# Libreria para hacer un pipeline de transformaciones
from sklearn.pipeline import Pipeline
# Libreria para juntar todos los pasos de preprocesamiento en una funcion de transformacion
from sklearn.compose import ColumnTransformer
# Importo una funcion que yo mismo cree en modulo de python que vive a la misma altura (en el mismo folder y mismo nivel) que mi notebook actual
from herramientas import borrar_columnas

## Carga de datos

In [4]:
#1) Leer datos de un archivo csv, usamos la libreria de pandas(pd)
train=pd.read_csv('../Datos/train.csv')

## Exploracion inicial de datos

In [3]:
#2) Conocer las variables con valores faltantes y el tipo de variable en el data set
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
#3)  Obtengo una lista manejable de las columnas del data frame
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [5]:
# 4) Separo mis variables independientes y mi variable de respuesta
#guardo una variable solo con mi variable dependiente
y=train['SalePrice']
#Tiro del dataframe mi variable dependiente y conservo las demas
train=train.drop('SalePrice', axis=1)

In [6]:
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

## Etapa de preprocesamiento para el modelado

In [7]:
# 4) Seleccion inicial de variables
# Elimino ciertas variables del data set que no empleare en este ejercicio por practicidad
#Agregamos de manera artificial una variable que no existe en el dataset para provar el comportamiento de la estructura try except:
# "ARTIFICIAL"
columnas_eliminar=["Id","LotShape","LandSlope","ExterQual","BsmtFinType1","BsmtFinType2","HeatingQC",
           "CentralAir","KitchenQual","Functional","FireplaceQu","GarageFinish","GarageQual",
           "GarageCond","PavedDrive","PoolQC","Fence","BsmtQual","BsmtCond","ExterCond","ExterQual","KitchenQual","ARTIFICIAL"]

In [8]:
# 5) Seleccion inicial de variables
# Uso el metodo pandas.drop para borrar columnas, establezco el eje en el cual borrare elementos (eje 1= columna).
try:
    train.drop(columnas_eliminar, axis=1)
except KeyError:
    print("Hay una columna que intentaste borrar y que no pertenece al df")

Hay una columna que intentaste borrar y que no pertenece al df


In [9]:
# 6) Evito los errores ocasiondos por KeyError usando una funcion que almacene en el modulo herramientas
# Asigno el dataframe al resultado de invocar la funcion que defini en mi modulo anterior
train=borrar_columnas(train,columnas_eliminar)

In [10]:
# 7) Reviso si mi funcion anterior me ayudo a eliminar las columnas que no quiero en el df
train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'Foundation', 'BsmtExposure',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageType',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscFeature',
       'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition'],
      dtype='object')

In [11]:
#Reviso si existen valores faltnates en mi data frame
# Imprimo un booleano que me diga si hay o no valores faltantes
print(train.isnull().values.any())
# Imprimo el numero de valores faltantes
print(train.isna().sum().sum())

True
3251


### Preprocesamiento con sklearn

#### 1) Defino Pipelines de preprocesamiento para mis distintos tipos de datos

In [12]:
#a) Hago listas que separan las variables dependiendo de su tipo

variables_numericas=["LotFrontage","LotArea","OverallQual","OverallCond","YearBuilt","YearRemodAdd",
           "MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF",
          "2ndFlrSF","LowQualFinSF","GrLivArea","BsmtFullBath","BsmtHalfBath","FullBath",
          "HalfBath","BedroomAbvGr","KitchenAbvGr","TotRmsAbvGrd","Fireplaces","GarageYrBlt",
          "GarageCars","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch",
           "ScreenPorch","PoolArea","MiscVal","YrSold","MoSold"]

variables_categoricas=["MSSubClass","MSZoning","Street","Alley","LandContour","Utilities",
            "LotConfig","Neighborhood","Condition1","Condition2","BldgType",
            "HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType",
            "Foundation","Heating","Electrical","GarageType","MiscFeature","SaleType","SaleCondition"]

In [13]:
#b) Establezco pasos de preprocesamiento con funciones de sklearn para cada tipo de datos en mi df

#Preprocesamiento para variables numericas
#Uso la funcion Pipeline para juntar distintos preprocesamientos en un unico paso, en este caso aplico Standar Scaler que va a estandarizar mis variables numericas
# y simple imputer, que va a imputar valores faltantes con la media de las observaciones. Por ahora uso los parametros de default de las funciones
procesamiento_numericas = Pipeline(
    steps=[("scaler", StandardScaler()),
           ("imputer",SimpleImputer())])

# Uso la funcion pipeline para aplicar un preprocesamiento a mis variables categoricas, en este caso solo aplico One Hote Encoder y establezo que las variables que no pasen por
# este preprocesamiento deben ser ignoradas, esto es importante para evitar que el transformador tenga problemas al preprocesar datos que no ha visto, por ejemplo, los
# datos del set de test
procesamiento_categoricas = Pipeline(
    steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))])

In [14]:
#c) En esta celda junto los pasos que estableci en la celda anterior y le indico a Column trnasformer que columnas van a recibir cada tipo de preprocesamiento
pasos_de_preprocesamiento = ColumnTransformer(
    transformers=[
        # "Nombre que asigno al paso de procesamiento", Nombre del pipeline, Lista de columnas a recibir ese preprocesamiento
        ("numericas", procesamiento_numericas, variables_numericas),
        # "Nombre que asigno al paso de procesamiento", Nombre del pipeline, Lista de columnas a recibir ese preprocesamiento
        ("categoricasl", procesamiento_categoricas, variables_categoricas)
        ],
    # Comportamiento que defino para las variables que no recibieron ningun tipo de preprocesamiento, en este caso elijo tirarlas
    remainder='drop'
    )

In [15]:
#d) finalmente aplico el preprocesameinto a mis datos, ya que en los pasos b y c unicamente define el preprocesamiento, pero no lo aplique. 
# Dado que se trata de mis datos de entrenamiento me gustaria que el procesamiento sea aprendido (fit) y que ademas de aprenderse se transformen los datos (transform)
# por lo cual aplico fit_transform a mis datos.
Datos_preprocesados=pasos_de_preprocesamiento.fit_transform(train)

In [22]:
Datos_preprocesados

<1460x221 sparse matrix of type '<class 'numpy.float64'>'
	with 86140 stored elements in Compressed Sparse Row format>

### Modelado de datos: regresion lineal

In [16]:
# A continuacion empleo regresion linear con sklearn para demostrar los pasos basicos de esta libreria, en iteraciones posteriorres agregare: 1) gridsearch, 
#2) mas modelos, 3) pipeline de modelado
#Entreno mi regresor con los datos de entrenamiento
regresion_lineal=LinearRegression().fit(Datos_preprocesados,y)

In [17]:
# Hago predicciones sobre mi set de test
#1) aplico el preprocesamiento de datos, en este caso, no quiero volver a entrenar el preprocesamiento, por lo tanto solo aplico el metodo transform
test=pd.read_csv('./Datos/test.csv')
Test_preprocesados=pasos_de_preprocesamiento.transform(test)

In [18]:
#2)Hago predicciones con mi modelo ya entrenado
predicciones=regresion_lineal.predict(Test_preprocesados)

In [19]:
#3) Mando mis predicciones a un archivo csv
predicciones.tofile('./Resultados/predicciones.csv', sep=",")